In [5]:
from argparse import Namespace
import xml.etree.ElementTree as ET
import os
from json import dumps
import xmltodict

In [6]:
def xml_to_json(xml_filepath):
    xml_filepath = os.path.expandvars(xml_filepath)
    with open(xml_filepath) as f:
        s = f.read()
    return xmltodict.parse(s)

lyt, lyp = map(xml_to_json,
               ['$PDK_ROOT/$PDK/libs.tech/klayout/tech/$PDK.lyt',
                '$PDK_ROOT/$PDK/libs.tech/klayout/tech/$PDK.lyp'])

In [7]:
from sqlalchemy import create_engine, Table, Column, ForeignKey, Integer, Boolean, Float, String, Text
from sqlalchemy.orm import declarative_base, relationship
from sqlalchemy.sql.expression import text

Base = declarative_base()

class Technology(Base):
    __tablename__     = 'technologies'
    id                = Column(Integer, primary_key=True)
    
    name              = Column(String(255), nullable=False, unique=True)
    description       = Column(Text, nullable=True)
    dbu               = Column(Float, nullable=False)
    add_other_layers  = Column(Boolean, nullable=False)
    
    writer_options    = relationship('WriterOptions', uselist=False, back_populates='technology')
    connections       = relationship('Connection', back_populates='technology')
    symbols           = relationship('Symbol', back_populates='technology')
    

class WriterOptions(Base):
    __tablename__    = 'writer_options'
    id               = Column(Integer, primary_key=True)
    
    libname          = Column(String(255), nullable=False)
    max_vertex_count = Column(Integer, nullable=False)
    write_timestamps = Column(Boolean, nullable=False)
    
    technology_id    = Column(Integer, ForeignKey('technologies.id'))
    technology       = relationship('Technology', back_populates='writer_options')

class Connection(Base):
    __tablename__ = 'connections'
    id            = Column(Integer, primary_key=True)
    
    a_symbol_id   = Column(Integer, ForeignKey('symbols.id'))
    a_symbol      = relationship('Symbol', foreign_keys=[a_symbol_id], back_populates='as_a_symbol_connections')
    
    via_symbol_id = Column(Integer, ForeignKey('symbols.id'))
    via_symbol    = relationship('Symbol', foreign_keys=[via_symbol_id], back_populates='as_via_symbol_connections')
    
    b_symbol_id   = Column(Integer, ForeignKey('symbols.id'))
    b_symbol      = relationship('Symbol', foreign_keys=[b_symbol_id], back_populates='as_b_symbol_connections')
    
    technology_id = Column(Integer, ForeignKey('technologies.id'))
    technology    = relationship('Technology', back_populates='connections')

SymbolPhysicalLayerPairs = Table(
    'symbol_physical_layer_pairs', 
    Base.metadata,
    Column('symbol_id', Integer, ForeignKey('symbols.id'), primary_key=True),
    Column('physical_layer_id', Integer, ForeignKey('physical_layers.id'), primary_key=True),
)

class Symbol(Base):
    __tablename__             = 'symbols'
    id                        = Column(Integer, primary_key=True)
    
    name                      = Column(String(255), nullable=False)
    
    technology_id             = Column(Integer, ForeignKey('technologies.id'))
    technology                = relationship('Technology', back_populates='symbols')
    
    as_a_symbol_connections   = relationship('Connection', foreign_keys=[Connection.a_symbol_id], back_populates='a_symbol')
    as_via_symbol_connections = relationship('Connection', foreign_keys=[Connection.via_symbol_id], back_populates='via_symbol')
    as_b_symbol_connections   = relationship('Connection', foreign_keys=[Connection.b_symbol_id], back_populates='b_symbol')

    physical_layers           = relationship('PhysicalLayer', secondary=SymbolPhysicalLayerPairs, back_populates='symbols')
    
class PhysicalLayer(Base):
    __tablename__ = 'physical_layers'
    id            = Column(Integer, primary_key=True)

    layer         = Column(Integer, nullable=False)
    datatype      = Column(Integer, nullable=False)
    negative      = Column(Boolean, server_default=text('0'), nullable=False)
    
    symbols       = relationship('Symbol', secondary=SymbolPhysicalLayerPairs, back_populates='physical_layers')

engine = create_engine('sqlite:///pdk.db', echo=True)
Base.metadata.create_all(engine)

2025-09-28 04:08:32,284 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-28 04:08:32,285 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("technologies")
2025-09-28 04:08:32,285 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-28 04:08:32,286 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("technologies")
2025-09-28 04:08:32,286 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-28 04:08:32,287 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("writer_options")
2025-09-28 04:08:32,287 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-28 04:08:32,287 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("writer_options")
2025-09-28 04:08:32,287 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-28 04:08:32,288 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("connections")
2025-09-28 04:08:32,288 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-28 04:08:32,289 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("connections")
2025-09-28 04:08:32,289 INFO sqlal

In [15]:
keys = list(lyt['technology'].keys())
keys

['name',
 'description',
 'group',
 'dbu',
 'base-path',
 'original-base-path',
 'layer-properties_file',
 'add-other-layers',
 'default-grids',
 'reader-options',
 'writer-options',
 'connectivity']

In [21]:
lyt['technology']['connectivity']

{'connection': ['poly,licon,li',
  'li,mcon,met1',
  'met1,via1,met2',
  'met2,via2,met3',
  'met3,via3,met4',
  'met4,via4,met5'],
 'symbols': ["capm='89/44'",
  "cap2m='97/44'",
  "poly='66/20+66/5-66/13-66/14-66/15'",
  "licon='66/44'",
  "li='67/20+67/5-67/13-67/14-67/15'",
  "mcon='67/44'",
  "met1='68/20+68/5-68/14-68/15'",
  "via1='68/44'",
  "met2='69/20+69/5-69/14-69/15'",
  "via2='69/44'",
  "met3='70/20+70/5-70/14-70/15'",
  "via3='70/44-capm'",
  "met4='71/20+71/5-71/14-71/15'",
  "via4='71/44-cap2m'",
  "met5='72/20+72/5-72/14-72/15'"]}

In [17]:
from collections.abc import Iterable
for key in keys:
    if isinstance(lyt['technology'][key], Iterable) and not isinstance(lyt['technology'][key], str):
        print(key)

reader-options
writer-options
connectivity


In [20]:
lyt['technology']['writer-options']

{'gds2': {'write-timestamps': 'true',
  'write-cell-properties': 'false',
  'write-file-properties': 'false',
  'no-zero-length-paths': 'false',
  'multi-xy-records': 'false',
  'max-vertex-count': '8000',
  'max-cellname-length': '32000',
  'libname': 'LIB'},
 'oasis': {'compression-level': '2',
  'write-cblocks': 'false',
  'strict-mode': 'false',
  'write-std-properties': '1',
  'subst-char': '*',
  'permissive': 'false'},
 'cif': [{'polygon-mode': '0'},
  {'dummy-calls': 'false', 'blank-separator': 'false'}],
 'mag': {'lambda': '0', 'tech': None, 'write-timestamp': 'true'}}

In [38]:
lyp['layer-properties'].keys()

dict_keys(['properties', 'name', 'custom-dither-pattern', 'custom-line-style'])

In [46]:
len(lyp['layer-properties']['properties'])

429

In [55]:
lyp['layer-properties']['properties'][5]

{'frame-color': '#ffbff2',
 'fill-color': '#ffbff2',
 'frame-brightness': '0',
 'fill-brightness': '0',
 'dither-pattern': 'C1',
 'line-style': 'C0',
 'valid': 'true',
 'visible': 'true',
 'transparent': 'false',
 'width': '1',
 'marked': 'false',
 'xfill': 'false',
 'animation': '0',
 'name': 'pwell.cut - 64/14',
 'source': '64/14@1'}

In [43]:
len(lyp['layer-properties']['name'])

TypeError: object of type 'NoneType' has no len()

In [42]:
len(lyp['layer-properties']['custom-dither-pattern'])

55

In [54]:
lyp['layer-properties']['custom-dither-pattern'][50]

{'pattern': {'line': ['*.........*.....',
   '.....*..........',
   '..*.........*...',
   '.......*........',
   '....*.........*.',
   '.*.......*......',
   '......*.........',
   '...*.......*....',
   '........*.......',
   '.....*.......*..',
   '*.........*.....',
   '.......*........',
   '..*.........*...',
   '.........*......',
   '....*.........*.',
   '...........*....']},
 'order': '51',
 'name': 'curve'}

In [47]:
len(lyp['layer-properties']['custom-line-style'])

8

In [51]:
lyp['layer-properties']['custom-line-style'][0]

{'pattern': '***', 'order': '1', 'name': 'solid'}